In [1]:
!pip install tensorflow

import os
import datetime
import sys
import random
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm
from itertools import chain

In [2]:
from sklearn.model_selection import train_test_split
import tensorflow.keras.backend as K
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.merge import concatenate
from keras.models import Model, load_model
from keras.layers import Input, Layer, InputSpec
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose, DepthwiseConv2D
from keras.layers.pooling import MaxPooling2D


2021-11-25 12:53:53.130858: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-25 12:53:53.130906: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
!pip install neptune-client neptune-tensorflow-keras

from neptune.new.integrations.tensorflow_keras import NeptuneCallback

# SESSION


In [4]:
import neptune.new as neptune

run = neptune.init(
    project="denissimo/UNET-X",
    tags=['unet-x', 'binary_ce'],
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJiYzYyNmYyMC02YmVkLTRjN2QtYmY5ZC00NDdmZDBhODMxNTIifQ==",
)  # your credentials


neptune_cbk = NeptuneCallback(run=run, base_namespace='metrics')

Info (NVML): NVML Shared Library Not Found. GPU usage metrics may not be reported. For more information, see https://docs-legacy.neptune.ai/logging-and-managing-experiment-results/logging-experiment-data.html#hardware-consumption 


https://app.neptune.ai/denissimo/UNET-X/e/UNET-23
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [5]:
CFG = {
    'dataset_path': './data'
}

IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 3
NUM_SAMPLES = 3075
NUM_CLASSES = 3

CLASS_WEIGHTS = np.array([0.0038977 , 0.08838018, 0.90772211])

## 1) Uploading data

In [6]:
y = np.load(f'{CFG["dataset_path"]}/y_train.npy')
x = np.load(f'{CFG["dataset_path"]}/x_train.npy')

slice_invalid = [950, 993, 994, 995, 996, 997, 998, 999]

x = np.delete(x, slice_invalid, axis=0)
y = np.delete(y, slice_invalid, axis=0)

## 2) Common code samples

### * Metrics

In [7]:
def apply_ohe(sample, index_shape, n_classes):
    new_shape = (*index_shape, n_classes)
    responce_tensor = np.zeros(new_shape)
    for img_class in range(n_classes):
        responce_tensor[:,:,:,img_class] = (sample == img_class).reshape(index_shape)
    return responce_tensor

In [8]:
y_ohe = apply_ohe(y, (NUM_SAMPLES, IMG_HEIGHT, IMG_WIDTH), 3)

In [9]:
def IoU(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)

def IoU_class_0(y_true, y_pred):
    y_true_f = K.flatten(y_true[:,:,:,0])
    y_pred_f = K.flatten(y_pred[:,:,:,0])
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)

def IoU_class_1(y_true, y_pred):
    y_true_f = K.flatten(y_true[:,:,:,1])
    y_pred_f = K.flatten(y_pred[:,:,:,1])
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)

def IoU_class_2(y_true, y_pred):
    y_true_f = K.flatten(y_true[:,:,:,2])
    y_pred_f = K.flatten(y_pred[:,:,:,2])
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)


In [10]:
def separableConv(x, channels):
    x=DepthwiseConv2D((3,3),padding="same")(x)
    x=Conv2D(channels, (1,1),padding="same", activation="relu")(x)
    return x

def multi_unetx_model(n_classes=4, IMG_HEIGHT=256, IMG_WIDTH=256, IMG_CHANNELS=1):
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    s = Lambda(lambda x: x / 255)(inputs)   #No need for this if we normalize our inputs beforehand

    conv_1_1 = separableConv(s, 16)
    conv_1_1 = Dropout(0.1) (conv_1_1)
    conv_1_2 = separableConv(conv_1_1, 16)
    pool_1 = MaxPooling2D(2)(conv_1_2)

    conv_2_1 = separableConv(pool_1, 32)
    conv_2_1 = Dropout(0.1) (conv_2_1)
    conv_2_2 = separableConv(conv_2_1, 32)
    pool_2 = MaxPooling2D(2)(conv_2_2)

    conv_3_1 = separableConv(pool_2, 64)
    conv_3_1 = Dropout(0.2) (conv_3_1)
    conv_3_2 = separableConv(conv_3_1, 64)
    pool_3 = MaxPooling2D(2)(conv_3_2)

    conv_4_1 = separableConv(pool_3, 128)
    conv_4_1 = Dropout(0.2) (conv_4_1)
    conv_4_2 = separableConv(conv_4_1, 128)
    pool_4 = MaxPooling2D(2)(conv_4_2)

    conv_5_1 = separableConv(pool_4, 256)
    conv_5_1 = Dropout(0.3) (conv_5_1)
    conv_5_2 = separableConv(conv_5_1, 256)

    up_1 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (conv_5_2) 
    conc_1 = concatenate([conv_4_2, up_1])
    conv_up_1_1 = separableConv(conc_1, 128)
    conv_up_1_1 = Dropout(0.2) (conv_up_1_1)
    conv_up_1_2 = separableConv(conv_up_1_1, 128)

    up_2 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (conv_up_1_2) 
    conc_2 = concatenate([conv_3_2, up_2])
    conv_up_2_1 = separableConv(conc_2, 64)
    conv_up_2_1 = Dropout(0.2) (conv_up_2_1)
    conv_up_2_2 = separableConv(conv_up_2_1, 64)

    up_3 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (conv_up_2_2)
    conc_3 = concatenate([conv_2_2, up_3])
    conv_up_3_1 = separableConv(conc_3, 32)
    conv_up_3_1 = Dropout(0.1) (conv_up_3_1)
    conv_up_3_2 = separableConv(conv_up_3_1, 32)

    up_4 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (conv_up_3_2)
    conc_4 = concatenate([conv_1_2, up_4])
    conv_up_4_1 = separableConv(conc_4, 16)
    conv_up_4_1 = Dropout(0.1) (conv_up_4_1)
    conv_up_4_2 = separableConv(conv_up_4_1, 16)
    outputs = Conv2D(n_classes, (1, 1), activation='sigmoid') (conv_up_4_2)
     
    model = Model(inputs=[inputs], outputs=[outputs])
    return model

In [11]:
def get_model():
    return multi_unetx_model(n_classes=3, IMG_HEIGHT=256, IMG_WIDTH=256, IMG_CHANNELS=3)

In [12]:
model = get_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[IoU, 'accuracy', IoU_class_0, IoU_class_1, IoU_class_2])
model.summary()

# model = tf.keras.models.load_model('unet_tasks_binary_ce.h5', 
#     custom_objects={
#         'IoU': IoU,
#         'IoU_class_0': IoU_class_0,
#         'IoU_class_1': IoU_class_1,
#         'IoU_class_2': IoU_class_2,
#     })
# model.compile(optimizer='adam', loss=custom_loss, metrics=[IoU, 'accuracy', IoU_class_0, IoU_class_1, IoU_class_2])

2021-11-25 12:54:04.290459: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-25 12:54:04.290512: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-25 12:54:04.290531: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (sample): /proc/driver/nvidia/version does not exist
2021-11-25 12:54:04.290791: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 256, 256, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
depthwise_conv2d (DepthwiseConv (None, 256, 256, 3)  30          lambda[0][0]                     
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 16) 64          depthwise_conv2d[0][0]           
______________________________________________________________________________________________

In [ ]:
run['model/method'] = 'adam'
run['model/loss'] = 'binary_crossentropy'
run['model/batchsize'] = 128
run['model/epochs'] = 200

In [ ]:
def train_model(model, X, Y, save_path, batch_size, epochs, model_name=""):
    model_name += datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    earlystopper = EarlyStopping(patience=50, verbose=1)
    checkpointer = ModelCheckpoint(save_path, verbose=1, save_best_only=False)

    log_dir = "./" + model_name

    x_train, x_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.1, random_state=1)

    results = model.fit(x_train, y_train, validation_data=(x_valid, y_valid), batch_size=batch_size, epochs=epochs,
                        callbacks=[earlystopper, checkpointer
                                 , neptune_cbk
                            ], workers=16)

    return results


In [ ]:
train_model(model, x, y_ohe, 'unetx_tasks_binary_ce.h5', 128, 200, 'fcn_new')

In [ ]:
run.stop()